<a href="https://colab.research.google.com/github/ispromadhka/MBZUAI-AI-serch-agent/blob/main/MBZUAI_agent_search_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
requirements = [
    "langchain>=0.1.0",
    "langchain-community>=0.0.10",
    "crawl4ai>=0.2.0",
    "faiss-cpu>=1.7.4",
    "psycopg2-binary>=2.9.9",
    "sqlalchemy>=2.0.0",
    "sentence-transformers>=2.2.2",
    "pydantic>=2.0.0",
    "python-dotenv>=1.0.0",
]

with open("requirements.txt", "w") as f:
    for req in requirements:
        f.write(req + "\n")

with open("requirements.txt", "r") as f:
    print(f.read())




!pip install -r requirements.txt
!pip install google-adk beautifulsoup4 requests crawl4ai

In [ ]:
!crawl4ai-setup

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/mbzuai_data.db')

In [ ]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,documents
1,sqlite_sequence


In [ ]:
table_name = 'documents'
df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
df.head()

,id,content,source,crawled_at
0,1,![MBZUAI Logo](https://staticcdn.mbzuai.ac.ae/...,https://mbzuai.ac.ae/,2025-10-30 09:55:50


In [ ]:
df["content"][0]

'# 502 Bad Gateway\n* * *\nMicrosoft-Azure-Application-Gateway/v2\n'

In [ ]:
from google.adk.agents import Agent

root_agent = Agent(
  name="hello_agent",
  model="gemini-1.5-pro-preview-0514",
  instruction="You are a friendly assistant that can answer user questions."
)

Основные технологии

    LangChain — оркестрация цепочек и работа с LLM, поддержка интеграций.

​

Crawl4AI — современный Python-инструмент для быстрых скраппинга сайтов (в том числе JavaScript), с удобной интеграцией с LangChain.

    FAISS — библиотека векторного поиска и индексации для быстрого семантического поиска по извлечённым данным.

Хранилище

    SQLite — реляционная база данных для хранения структурированных данных:

    text
    [информация] | [ссылка на источник] | [дата публикации]

Вспомогательные библиотеки

    Requests или httpx — для подгрузки веб-страниц (например, если требуется получение данных вне Crawl4AI).

    Sentence Transformers (например, all-MiniLM-L6-v2) — для генерации эмбеддингов, если используются локальные модели.

Краткая схема архитектуры

    LangChain оркестрирует агентов, вызывая Crawl4AI для поиска и загрузки информации.

    Crawl4AI скачивает и препроцессит контент (Markdown/HTML).

    Извлечённый и очищенный текст проходит через LLM (open-source или self-hosted), чтобы структурировать данные.

    Данные сохраняются в PostgreSQL и индексируются в FAISS для быстрого поиска.

    Агент отвечает на запросы, комбинируя поиск по PostgreSQL (структурированные поля) и FAISS (семантика).


In [ ]:
!pip install langchain-google-genai pydantic --upgrade
!pip install --upgrade langchain langchain-core langchain-community langchain-google-genai

In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from typing import List

class AnalysisResult(BaseModel):
    """Структурированный результат анализа веб-страницы."""
    has_useful_info: bool = Field(description="Истина, если на странице есть полезная информация о MBZUAI.")
    summary: str = Field(description="Краткое содержание полезной информации о MBZUAI на странице (2-3 предложения).")
    useful_links: List[str] = Field(description="Список ПОЛНЫХ URL-адресов (начинающихся с http/https), которые ведут на страницы о поступлении, работе, исследованиях или сотрудничестве с MBZUAI.")

class LLMAnalyzer:
    """
    Анализирует контент страницы с помощью Google Gemini для извлечения
    полезной информации и ссылок.
    """
    def __init__(self):
        self.model = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            temperature=0.3,
            convert_system_message_to_human=True
        )
        self.parser = PydanticOutputParser(pydantic_object=AnalysisResult)
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", """Ты — умный ассистент-исследователь. Твоя задача — проанализировать текст с веб-страницы и извлечь информацию, связанную с университетом MBZUAI.

Критерии "полезной информации":
- Условия поступления (admissions)
- Вакансии и карьера (jobs, careers)
- Научные исследования и публикации
- Партнерства и сотрудничество (collaboration), особенно с MIT.

Отформатируй свой ответ строго в формате JSON, как указано ниже.
{format_instructions}"""),
                ("human", "Вот текст со страницы {page_content}"),
            ]
        ).partial(format_instructions=self.parser.get_format_instructions())
        self.chain = self.prompt | self.model | self.parser

    def analyze(self, page_content: str) -> AnalysisResult:
        """Анализирует текст и возвращает структурированный результат."""
        try:
            return self.chain.invoke({"page_content": page_content[:30000]})
        except Exception as e:
            print(f"Ошибка при анализе LLM: {e}")
            return AnalysisResult(has_useful_info=False, summary="", useful_links=[])

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('/content/mbzuai__research.db')

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
# tables

table_name = 'documents'
df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
df.head()

,id,content,source,crawled_at
0,1,Веб-страница MBZUAI содержит обширную информац...,https://mbzuai.ac.ae/,2025-10-30 11:12:59
1,2,"Несмотря на то, что страница является сообщени...",https://mbzuai.ac.ae/admissions/,2025-10-30 11:13:26
2,3,MBZUAI является исследовательским университето...,https://mbzuai.ac.ae/research/,2025-10-30 11:13:52
3,4,Университет MBZUAI предлагает программы бакала...,https://en.wikipedia.org/wiki/Mohamed_bin_Zaye...,2025-10-30 11:15:21
4,5,Данная страница подробно описывает процесс пос...,https://mbzuai.ac.ae/study/admission-process/,2025-10-30 11:15:46


# Final

In [ ]:
import sqlite3
import time
import asyncio
import nest_asyncio
from typing import List

from crawl4ai import AsyncWebCrawler
from langchain_community.docstore.document import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

nest_asyncio.apply()


class AnalysisResult(BaseModel):
    """Структурированный результат анализа веб-страницы."""
    has_useful_info: bool = Field(description="Истина, если на странице есть полезная информация о MBZUAI.")
    summary: str = Field(description="Краткое содержание полезной информации о MBZUAI на странице (2-3 предложения).")
    useful_links: List[str] = Field(description="Список ПОЛНЫХ URL-адресов (начинающихся с http/https), которые ведут на страницы о поступлении, работе, исследованиях или сотрудничестве с MBZUAI.")

class LLMAnalyzer:
    """Анализирует контент страницы с помощью Google Gemini."""
    def __init__(self):
        self.model = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            temperature=0.3, convert_system_message_to_human=True)
        self.parser = PydanticOutputParser(pydantic_object=AnalysisResult)
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """Ты — умный ассистент-исследователь. Твоя задача — проанализировать текст с веб-страницы и извлечь информацию, связанную с университетом MBZUAI.
Критерии "полезной информации":
- Условия поступления (admissions)
- Вакансии и карьера (jobs, careers)
- Научные исследования и публикации
- Партнерства и сотрудничество (collaboration), особенно с MIT.
Отформатируй свой ответ строго в формате JSON, как указано ниже.
{format_instructions}"""),
            ("human", "Вот текст со страницы {page_content}"),
        ]).partial(format_instructions=self.parser.get_format_instructions())
        self.chain = self.prompt | self.model | self.parser

    def analyze(self, page_content: str) -> AnalysisResult:
        """Анализирует текст и возвращает структурированный результат."""
        try:
            return self.chain.with_config(timeout=60).invoke({"page_content": page_content[:30000]})
        except Exception as e:
            print(f"Ошибка при анализе LLM (возможно, таймаут): {e}")
            return AnalysisResult(has_useful_info=False, summary="", useful_links=[])


class DataCollector:
    """Отвечает за скрапинг веб-страниц и сохранение в БД."""
    def __init__(self, db_path: str):
        self.db_path = db_path
        self._setup_database()

    def _setup_database(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY AUTOINCREMENT, content TEXT NOT NULL, source TEXT NOT NULL UNIQUE, crawled_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )""")
        conn.commit()
        conn.close()

    async def acrawl_website(self, crawler: AsyncWebCrawler, url: str) -> list[Document]:
        documents = []
        try:
            result = await crawler.arun(url=url, timeout=100)
            if result and result.markdown and "Bad Gateway" not in result.markdown:
                documents.append(Document(page_content=result.markdown, metadata={"source": result.url}))
        except asyncio.TimeoutError:
            print(f"TimeoutError: Скрапинг {url} занял слишком много времени и был пропущен.")
        except Exception as e:
            print(f"Ошибка при скрапинге {url}: {e}")
        return documents

    def save_to_db(self, documents: list[Document]):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        saved_count = 0
        for doc in documents:
            try:
                cursor.execute("INSERT INTO documents (content, source) VALUES (?, ?)", (doc.page_content, doc.metadata['source']))
                saved_count += 1
            except sqlite3.IntegrityError:
                pass
        conn.commit()
        conn.close()
        if saved_count > 0:
            print(f"Сохранено {saved_count} новых записей в БД.")


class ResearchAgent:
    """Оркестрирует процесс поиска, сбора и анализа информации."""
    def __init__(self, data_collector: DataCollector, llm_analyzer: LLMAnalyzer, seed_urls: List[str]):
        self.collector = data_collector
        self.analyzer = llm_analyzer
        self.urls_to_crawl = list(seed_urls)
        self.visited_urls = set()
        self.processed_count = 0

    async def _process_single_url(self, crawler: AsyncWebCrawler):
        if not self.urls_to_crawl: return
        url = self.urls_to_crawl.pop(0)
        if url in self.visited_urls: return

        self.processed_count += 1
        print(f"\n[Страница {self.processed_count}] Обрабатываю: {url}")
        self.visited_urls.add(url)

        docs = await self.collector.acrawl_website(crawler, url)
        if not docs: return

        page_content = docs[0].page_content
        print("Анализирую контент с помощью Gemini...")
        analysis_result = self.analyzer.analyze(page_content)

        if analysis_result.has_useful_info:
            print(f"✅ Найдена полезная информация. Сохраняю в БД.")
            summary_doc = Document(page_content=analysis_result.summary, metadata={"source": url})
            self.collector.save_to_db([summary_doc])

            if analysis_result.useful_links:
                new_links_count = 0
                for new_link in analysis_result.useful_links:
                    if new_link not in self.visited_urls and new_link not in self.urls_to_crawl:
                        self.urls_to_crawl.append(new_link)
                        new_links_count += 1
                if new_links_count > 0:
                    print(f"🔎 Найдено и добавлено {new_links_count} уникальных ссылок. Всего в очереди: {len(self.urls_to_crawl)}.")
        else:
            print("❌ Полезной информации не найдено, пропускаю.")

    async def run_research_mission(self, mode: str = 'auto', batch_size: int = 5):
        print(f"--- Запускаю миссию в режиме: {mode.upper()} ---")
        try:
            async with AsyncWebCrawler(headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}) as crawler:
                if mode == 'auto':
                    while self.urls_to_crawl:
                        await self._process_single_url(crawler)
                elif mode == 'manual':
                    while self.urls_to_crawl:
                        for _ in range(batch_size):
                            if not self.urls_to_crawl: break
                            await self._process_single_url(crawler)
                        if not self.urls_to_crawl: break
                        user_input = input(f"\n--- Обработано {self.processed_count} стр. В очереди {len(self.urls_to_crawl)}. Продолжить? (y/n): ")
                        if user_input.lower() not in ['y', 'yes', 'д', 'да']:
                            print("Останавливаю миссию по команде пользователя.")
                            break
        except KeyboardInterrupt:
            print("\n--- Миссия прервана пользователем (KeyboardInterrupt) ---")
        except Exception as e:
            print(f"\n--- Произошла критическая ошибка. Завершаю миссию: {e} ---")

        print("\n--- Миссия по исследованию завершена ---")
        print(f"Всего обработано: {self.processed_count} страниц.")


async def main():
    EXECUTION_MODE = "auto"
    BATCH_SIZE_FOR_MANUAL_MODE = 10
    DB_FILE = "mbzuai_agent_research.db"

    seed_urls = [
        "https://mbzuai.ac.ae/",
        "https://mbzuai.ac.ae/admissions/",
        "https://mbzuai.ac.ae/research/",
        "https://www.timeshighereducation.com/world-university-rankings/mohamed-bin-zayed-university-artificial-intelligence-mbzuai",
        "https://en.wikipedia.org/wiki/Mohamed_bin_Zayed_University_of_Artificial_Intelligence"
    ]

    data_collector = DataCollector(db_path=DB_FILE)
    llm_analyzer = LLMAnalyzer()
    agent = ResearchAgent(data_collector, llm_analyzer, seed_urls)

    await agent.run_research_mission(mode=EXECUTION_MODE, batch_size=BATCH_SIZE_FOR_MANUAL_MODE)

asyncio.run(main())

--- Запускаю миссию в режиме: AUTO ---


[INIT].... → Crawl4AI 0.7.6 


[Страница 1] Обрабатываю: https://mbzuai.ac.ae/


[FETCH]... ↓ https://mbzuai.ac.ae/                                                                                |
✓ | ⏱: 6.11s 

[SCRAPE].. ◆ https://mbzuai.ac.ae/                                                                                |
✓ | ⏱: 0.46s 

...

[COMPLETE] ● https://academy.mbzuai.ac.ae/training-programs/mep-accelerated/                                      |
✓ | ⏱: 5.52s 

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 31.693259187s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, retry_delay {
  seconds: 31
}
].


Анализирую контент с помощью Gemini...


* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 29.574105448s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, retry_delay {
  seconds: 29
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 25.436966456s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_i

Ошибка при анализе LLM (возможно, таймаут): 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250
Please retry in 29.022511043s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, retry_delay {
  seconds: 29
}
]
❌ Полезной информации не найдено, пропускаю.

--- Миссия по исследованию завер